In [1]:
import os
import cv2 as cv2
import numpy as np

# Function to apply padding around the image
def pad_image(image, padding=20):
    # Padding with reflection, to avoid black borders
    padded_image = cv2.copyMakeBorder(image, padding, padding, padding, padding, cv2.BORDER_REFLECT)
    return padded_image

# Function to apply scaling (zoom in or zoom out)
def scale_image(image, scale_factor):
    rows, cols = image.shape[:2]
    scaled_image = cv2.resize(image, None, fx=scale_factor, fy=scale_factor)
    
    # Crop or pad to retain the original size after scaling
    if scale_factor > 1:
        # If zooming in, crop the central region
        startx = (scaled_image.shape[1] - cols) // 2
        starty = (scaled_image.shape[0] - rows) // 2
        scaled_image = scaled_image[starty:starty+rows, startx:startx+cols]
    else:
        # If zooming out, pad to return to the original size
        border_x = (cols - scaled_image.shape[1]) // 2
        border_y = (rows - scaled_image.shape[0]) // 2
        scaled_image = cv2.copyMakeBorder(scaled_image, border_y, border_y, border_x, border_x, cv2.BORDER_REFLECT)
    
    return scaled_image

# Function to apply warping (deforming regions of the face)
def warp_image(image):
    rows, cols = image.shape[:2]
    
    # Points for the affine transform (focus on facial regions like eyes, mouth)
    src_points = np.float32([[cols * 0.3, rows * 0.4], [cols * 0.7, rows * 0.4], [cols * 0.5, rows * 0.6]])
    dst_points = np.float32([[cols * 0.3, rows * 0.42], [cols * 0.68, rows * 0.38], [cols * 0.52, rows * 0.62]])
    
    # Generate the affine transformation matrix
    warp_matrix = cv2.getAffineTransform(src_points, dst_points)
    
    # Apply warping
    warped_image = cv2.warpAffine(image, warp_matrix, (cols, rows))
    
    return warped_image

# Process images in the directory
def process_images_in_directory(input_dir, output_dir, padding=20):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for filename in os.listdir(input_dir):
        img_path = os.path.join(input_dir, filename)
        image = cv2.imread(img_path)
        
        if image is not None:
            #Add padding to avoid black pixels
            padded_image = pad_image(image, padding)
            
            # Step 2: Apply scaling (zoom in or zoom out)
            scaled_image = scale_image(padded_image, scale_factor=1.05)  # 5% zoom in
            
            # Step 3: Apply warping (subtle facial deformation)
            warped_image = warp_image(scaled_image)
            
            # Step 4: Crop back to original size (remove the padding)
            cropped_image = warped_image[padding:-padding, padding:-padding]
            
            # Save the transformed image
            output_path = os.path.join(output_dir, 'transformed_' + filename)
            cv2.imwrite(output_path, cropped_image)

# Example usage:
input_directory = 'C:/Users/HPA02532Y/Documents/GitHub/Face-Expression-Change-Detection/Dataset/anger'
output_directory = 'C:/Users/HPA02532Y/Documents/GitHub/Face-Expression-Change-Detection/Dataset/newanger'
process_images_in_directory(input_directory, output_directory, padding=20)
